# Remote Controlled Car


In [5]:
%serialconnect

serial exception on close write failed: [Errno 6] Device not configured
Found serial ports: /dev/cu.SLAB_USBtoUART, /dev/cu.usbserial-0001, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.OnkyoCR-265 
Connecting to --port=/dev/cu.SLAB_USBtoUART --baud=115200 
Ready.


In [6]:
import uasyncio as asyncio
import aioble
from bluetooth import UUID
import struct

# Because MicroPython doesn't even support fucking enums :(
NoAcceleration = 0
ForwardAcceleration = 1
BackwardAcceleration = 2

class BLESteeringServer:
    # For further information, see https://developer.nordicsemi.com/nRF5_SDK/nRF51_SDK_v4.x.x/doc/html/group___b_l_e___a_p_p_e_a_r_a_n_c_e_s.html
    GATT_APPEARANCE = 128
    NAME = "Remote Car"

    STEERING_SERVICE_UUID = UUID("B742662E-6D94-43BD-B257-8077D259EE5E")
    ROTATION_CHARACTERISTIC_UUID = UUID("16623586-A80C-4092-8042-652F934B8167")
    ACCELERATION_CHARACTERISTIC_UUID = UUID("D4C3EA03-F05A-41B2-B8AA-B5C8DB0ACD26")

    ADVERTISING_INTERVAL_MILLISECONDS = 250_000

    def __init__(self):
        self.deviceConnected = False
        self.connection = None

        self.gattSteeringService = aioble.Service(self.STEERING_SERVICE_UUID)
        self.registerCharacteristics()


    def registerCharacteristics(self):
        self.rotationCharacteristic = aioble.Characteristic(
            self.gattSteeringService,
            self.ROTATION_CHARACTERISTIC_UUID,
            read=True,
            write=True
        )

        self.accelerationCharacteristic = aioble.Characteristic(
            self.gattSteeringService,
            self.ACCELERATION_CHARACTERISTIC_UUID,
            read=True,
            write=True
        )

        aioble.register_services(self.gattSteeringService)

    # Because of "reasons", once connected to a central device, self.connection.is_connected() will never return False, even when the device is already disconnected
    # This represents a crutial part of the python community. Everyone botches the hell out of it and nothing works as it should...
    def isConnected(self) -> bool:
        if self.connection is not None:
            test = self.connection.is_connected()
            return test
        
        return False
    
    
    # Julian: And now we return an enum to indicates the direction
    # Python: Enums??!! Nope!
    # Julian: But that's good code style, it makes the code way more readable
    # Python: Fuck you and your fancy code styles, just return an integer and let the user guess what values are possible
    def getAccelerationDirection(self) -> int:
        return int.from_bytes(self.accelerationCharacteristic.read(), "little")
    
    
    def getRotation(self) -> int:
        return (int.from_bytes(self.rotationCharacteristic.read(), "little") - 100) * -1

    async def advertiseAndWaitForConnection(self):
        self.deviceConnected = self.isConnected()

        if self.deviceConnected is False:
            self.connection = await aioble.advertise(
                self.ADVERTISING_INTERVAL_MILLISECONDS,
                name=self.NAME,
                services=[self.STEERING_SERVICE_UUID],
                appearance=self.GATT_APPEARANCE,
            )
            self.deviceConnected = True

In [7]:
# https://github.com/a-fuchs/WPFInformatik/blob/main/jupyter/300_PowerAmplifier_L9110/SimpleCar.ipynb

import machine

class Motor :
    def __init__( self, 
            pinA, 
            pinB,
            frequency     = 1000,
            dutyMinLeft   =    0,  # Duty for speed = 0:              motor turns left
            dutyMaxLeft   = 1023,  # Duty for speed = fSpeedMaxLeft:  motor turns left
            dutyMinRight  =    0,  # Duty for speed = 0:              motor turns right
            dutyMaxRight  = 1023,  # Duty for speed = fSpeedMaxRight: motor turns right
            speedMaxLeft  = -1.0,  # Minimal value for setSpeed() --> iDutyMaxLeft
            speedMaxRight =  1.0   # Maximal value for setSpeed() --> iDutyMaxRight
        ):
        self._iFrequency    = frequency
        self._fScaleLeft    = (dutyMaxLeft  - dutyMinLeft )/speedMaxLeft
        self._fScaleRight   = (dutyMaxRight - dutyMinRight)/speedMaxRight
        self._iDutyMinLeft  = dutyMinLeft
        self._iDutyMaxLeft  = dutyMaxLeft
        self._iDutyMinRight = dutyMinRight
        self._iDutyMaxRight = dutyMaxRight
        
        self._fCurSpeed     = 0
        
        self._pwmA = machine.PWM(machine.Pin(pinA), freq=self._iFrequency, duty=0)
        self._pwmB = machine.PWM(machine.Pin(pinB), freq=self._iFrequency, duty=0)
        
        self._pwmDir   = self._pwmA
        self._pwmSpeed = self._pwmB
        
    def speedToDuty(self, _fSpeed):
        if _fSpeed == 0:
            return 0
        elif _fSpeed < 0:
            return int(self._iDutyMinLeft  + _fSpeed * self._fScaleLeft + 0.5)
        else :
            return int(self._iDutyMinRight + _fSpeed * self._fScaleRight + 0.5)
            
    
    def setLeft(self):
        self._pwmDir = self._pwmA
        self._pwmDir.duty(0)
        self._pwmSpeed = self._pwmB
    
    def setRight(self):
        self._pwmDir   = self._pwmB
        self._pwmDir.duty(0)
        self._pwmSpeed = self._pwmA
    
    def setSpeed(self, _fSpeed):
        if _fSpeed != self._fCurSpeed:
            iDuty = self.speedToDuty(_fSpeed)
                
            if _fSpeed > 0 and self._fCurSpeed <= 0 :
                self.setLeft()

            elif _fSpeed < 0 and self._fCurSpeed >= 0 :
                self.setRight()
        
            self._pwmSpeed.duty(iDuty)
        
        self._fCurSpeed = _fSpeed

In [8]:
import display

screen = display.Display()

leftMotor = Motor(
    14,
    26,
    frequency = 300,
    dutyMinLeft = 0,
    dutyMaxLeft = 1023,
    dutyMinRight = 0,
    dutyMaxRight = 1023,
    speedMaxLeft = -1.0,
    speedMaxRight =  1.0
)

rightMotor = Motor(
    13,
    12,
    frequency = 300,
    dutyMinLeft = 0,
    dutyMaxLeft = 1023,
    dutyMinRight = 0,
    dutyMaxRight = 1023,
    speedMaxLeft = -1.0,
    speedMaxRight =  1.0
)

bleServer = BLESteeringServer()

# In real programming languages, the "direction" parameter wouldn't be an integer, but rather an enum.
def getSpeed(left: bool, rotation: int, direction: int) -> int:
    directionMultiplier = 1 if direction is ForwardAcceleration else -1

    if left and rotation <= 0:
        return (1 - (abs(rotation) / 100)) * directionMultiplier

    elif left and rotation >= 0:
        return 1 * directionMultiplier
    
    elif not left and rotation > 0:
        return (1 - (abs(rotation) / 100)) * directionMultiplier
    
    else:
        return 1 * directionMultiplier


# Because of "reasons", this program will hang indefinitely, once the BLE connection is lost...
# Great BLE library! -1 out of 10 stars for functionality, but lets give them 5 stars for their effort.
async def main():
    global screen, bleServer, leftMotor

    while True:
        if not bleServer.isConnected():
            screen.text("Waiting for", 0, 0)
            screen.text("connection", 0, 20)
            screen.show()

        await bleServer.advertiseAndWaitForConnection()

        currentAccelerationDirection = bleServer.getAccelerationDirection()
        currentRotation = bleServer.getRotation()

        if currentAccelerationDirection is not ForwardAcceleration and not BackwardAcceleration:
            leftMotor.setLeft(0)
            rightMotor.setSpeed(0)
        else:
            leftSpeed = getSpeed(True, currentRotation, currentAccelerationDirection)
            rightSpeed = getSpeed(False, currentRotation, currentAccelerationDirection)
            
            screen.clear()
            screen.text("Left: " + str(leftSpeed), 0, 0)
            screen.text("Right: " + str(rightSpeed), 0, 20)

            leftMotor.setSpeed(leftSpeed)
            rightMotor.setSpeed(rightSpeed)

        screen.show()



asyncio.run(main())

......................................................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 86, in <module>
  File "uasyncio/core.py", line 1, in run
  File "uasyncio/core.py", line 1, in run_until_complete
  File "<stdin>", line 82, in main
KeyboardInterrupt: 


In [9]:
#%fetchfile --load /main.py

from machine import Pin,I2C
import time
import ssd1306
try:
 i2c=I2C(1,scl=Pin(15),sda=Pin(4))
 d=ssd1306.SSD1306_I2C(128,64,i2c)
 d.fill(0)
 d.invert(0)
 d.text("Esp32 HelTec",17,23,1)
 d.text("OLED 128x64",20,37,1)
 d.show()
 for i in range(5):
  time.sleep_ms(200)
  d.invert(i%2)
 for i in range(45):
  d.scroll(0, -1)
  time.sleep_ms(10)
  d.show()
except:
 pass
finally:
 try: del d
 except: pass
 try: del i2c
 except: pass
 import gc
 gc.collect()

Fetched 455=455 bytes from /main.py.
